# Data Scientist (DS)

Outline of what the DS will do

1. DS logs into DO1 and DO2's datasites as guest  
2. DS explores the datasets  
3. DS prepares syft_flwr code  
4. DS bootstraps the syft_flwr project  
5. DS runs flwr and syft_flwr simulations (optional)   
6. DS submits jobs to the DOs' datasites  
7. DS starts the FL server code  
8. DS Observes the Results

## 1. DS logs into DO1 and DO2's datasites as guest


DO will also have their own datasite where they can log in as an admin, but it's not needed in this workflow

<img src="../images/dsLogsInAsGuests.png" width="55%" alt="DS logs into DOs' datasites as guests">

In [ ]:
import os
from pathlib import Path

from loguru import logger
from syft_rds.orchestra import setup_rds_server

DS = "ds@openmined.org"
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"

ds_stack = setup_rds_server(email=DS, root_dir=Path("."), key="local_syftbox_network")
do1_guest = ds_stack.init_session(host=DO1)
do2_guest = ds_stack.init_session(host=DO2)

In [ ]:
ds = ds_stack.client

In [ ]:
do1_guest.is_admin

In [ ]:
do2_guest.is_admin

Set some constants and variables

In [ ]:
os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(ds_stack.client.config_path)
os.environ["LOGURU_LEVEL"] = "DEBUG"
os.environ["SYFT_FLWR_MSG_TIMEOUT"] = "60"

do_clients = [do1_guest, do2_guest]
do_emails = [DO1, DO2]

## 2. DS explores the datasets

DS can access the DOs' mock data, but can't access the private data

<img src="../images/dsExploresDOsDatasets.png" width="71%" alt="DS explores datasets">

In [ ]:
mock_paths = []
for client in do_clients:
    dataset = client.dataset.get_all()[0]
    logger.info(f"Client {client.host}'s dataset name: {dataset.name}")
    mock_paths.append(dataset.get_mock_path())

Check that the DS can't access the private datasets, even with local test

In [ ]:
try:
    dataset.get_private_path()
except Exception as e:
    logger.error(e)

## 3. DS prepares `syft_flwr` code

A `syft_flwr` project requires minimal changes to a Flower project:

1. The `syft_flwr` fedrag project has the same structure as an [Flower fedrag project](https://flower.ai/docs/examples/fedrag.html/):
```
fedrag
├── fedrag/
│   ├── __init__.py
│   ├── client_app.py
│   ├── llm_querier.py
│   ├── mirage_qa.py
│   ├── retriever.py
│   ├── retriever.yaml
│   ├── server_app.py
│   └── task.py
├── pyproject.toml
└── README.md
``` 

Concretely, the `fedrag` project code is at `../fedrag` 

<img src="../images/dsPreparesSyftFlwrProject.png" width="20%" alt="DS prepares code">


where the DS will run the `ServerApp` (defined in `server_app.py`), and the `ClientApp` (in `client_app.py`)

<img src="../images/dsSyftFlwrProjectArch.png" width="69%" alt="syft_flwr project architecture">


In [ ]:
SYFT_FLWR_PROJECT_PATH = Path("fedrag_v1")
assert SYFT_FLWR_PROJECT_PATH.exists()

2. Compared to a Flower project, the DS only needs to change a bit of how the `client_app.py`, which runs on the data owners' datasites, loads the data:

<img src="../images/dsCodeLoadsClientData.png" width="70%" alt="load datasets comparison">


## 4. DS bootstraps the `syft_flwr` project

After the DS has prepared the `syft_flwr` project code, they can run the `syft_flwr.bootstrap` command to modify the project's metadata in `pyproject.toml` that specifies the unique name of the FL run, who are the data owners and who is the aggregator
```
[tool.syft_flwr]
app_name = "ds@openmined.org_fed-analytics-diabetes_1753435643"
datasites = [
    "do1@openmined.org",
    "do2@openmined.org",
]
aggregator = "ds@openmined.org"
```
and creates the `main.py` to specifies who runs what based on the new metatdata information

In [ ]:
import syft_flwr

try:
    !rm -rf {SYFT_FLWR_PROJECT_PATH / "main.py"}
    syft_flwr.bootstrap(SYFT_FLWR_PROJECT_PATH, aggregator=DS, datasites=do_emails)
    logger.info("Bootstrapped project successfully ✅")
except Exception as e:
    logger.error(e)

## 5. DS runs `flwr` and `syft_flwr` simulations (optional)

In [ ]:
# clean up
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fedrag" / "**/__pycache__"}  # clean before submitting

Now the DS can run `syft_flwr` simulation on DO's mock data by launching 2 threads that run DOs code and a thread that runs the DS code 

<img src="../images/dsRunsSyftFLWRSimulation.png" width="53%" alt="ds runs simulation with syft_flwr.run">

In [ ]:
RUN_SIMULATION = True

if RUN_SIMULATION:
    logger.info(f"running syft_flwr simulation with mock paths: {mock_paths}")
    syft_flwr.run(SYFT_FLWR_PROJECT_PATH, mock_paths)

Please look into the dir pointed to by `📝 Log directory` to see the logs of different simulated clients, by default it stays in the `fed-analytics-diabetes/simulation_logs`

<img src="../images/dsSimulationLogs.png" width="20%" alt="DS simulation logs folder">

We will also see that the aggregated historgrams are saved under `figures/` dir 

<img src="../images/dsAggregatedFiguresFolder.png" width="20%" alt="DS aggregated figures folder">

However, since running simulation is done on mock data, the local frequency is very low (only 7 as maximum for age)

<img src="../images/dsLocalFrequencyMock.png" width="50%" alt="local frequency mock data">

## 6. DS submits jobs to the DOs' datasites

The DS submits the code in `fed-analytics-diabetes` to the DO's datasites

<img src="../images/dsSubmitsJobs.png" width="50%" alt="DS submits jobs">

In [ ]:
# clean up before submitting jobs
!rm -rf {SYFT_FLWR_PROJECT_PATH / "fedrag" / "__pycache__"}
!rm -rf {SYFT_FLWR_PROJECT_PATH / "simulation_logs"}

In [ ]:
logger.info(f"sending job to {do1_guest.host}")
job = do1_guest.job.submit(
    name="fedrag",
    user_code_path=SYFT_FLWR_PROJECT_PATH,
    dataset_name="statpearls",
    entrypoint="main.py",
)
logger.success(job)

In [ ]:
logger.info(f"sending job to {do2_guest.host}")
job = do2_guest.job.submit(
    name="fedrag",
    user_code_path=SYFT_FLWR_PROJECT_PATH,
    dataset_name="textbooks",
    entrypoint="main.py",
)
logger.success(job)

<img src="../images/dsWaitsForJobsToBeApproved.png" width="30%" alt="DS waits for jobs to be approved">

## 7. DS starts the FL server code

By running the FL server code, the DS aggregates the models trained on DOs' private local data into an improved global model in multiple rounds

<video width="90%" controls>
  <source src="../images/fed-analytics.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [ ]:
!uv run {str(SYFT_FLWR_PROJECT_PATH / "main.py")} --active

In [ ]:
!rm -rf local_syftbox_network/ .syftbox/ .server/

## 8. DS Observes the Results

Now the DS can monitor the aggregated figures trained on DO's private datasets in the `figures` folder

<img src="../images/dsAggregatedFiguresFolder.png" width="20%" alt="DS aggregated figures folder">

Now, the local frequency when running job on private data is much higher compared to running simulation on mock data

<img src="../images/dsLocalFrequencyReal.png" width="50%" alt="DS aggregated frequency real">